In [133]:
from text_retriever_class import TextRetriever

semantic = TextRetriever(print_categories=False)

In [134]:
question

"What are my core clases? I'm a math major."

In [141]:
%%timeit

semantic.retrieve(
    question=question,
    type="semantic",
    use_classifier=False,
    top_n=1
)

49.5 ms ± 6.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [130]:
semantic.retrieve(
    question=question,
    type="semantic_rerank",
    use_classifier=False,
    top_n=1
)

Selected 'no categorization' - continuing with searching all categories...


'To complete a major in mathematics, students take nine required courses that provide a thorough foundation in several central areas of mathematics, a course in structured programming, and a minimum of three advanced courses chosen to give special depth in at least one area.'

In [131]:
semantic.retrieve(
    question=question,
    type="semantic_rerank",
    use_classifier=True,
    top_n=1
)

{'main_categories': ['academics'], 'sub_categories': ['academics|mathematics']}


'To complete a major in mathematics, students take nine required courses that provide a thorough foundation in several central areas of mathematics, a course in structured programming, and a minimum of three advanced courses chosen to give special depth in at least one area.'

In [18]:
from text_retriever_class import TypesenseRetriever

typesense = TypesenseRetriever(print_categories=True)

Docker container not found...
Docker container is not running. Starting it now...
Docker container started.



In [24]:
question="How can I apply to Brockport for an undergraduate degree?"
alpha=1

In [32]:
typesense.retrieve(
    question=question,
    top_n=1,
    alpha=0.8,
    use_classifier=True
)

{'main_categories': ['admissions'], 'sub_categories': []}


'Brockport operates on a rolling admissions basis, which means we make decisions on applications as they become complete. First-time students who meet academic and income guidelines are eligible to apply.'

In [128]:
typesense.retrieve(
    question=question,
    top_n=1,
    alpha=alpha,
    use_classifier=True
)

{'main_categories': ['academics'], 'sub_categories': ['academics|mathematics']}


'Our program meets the needs of a broad range of students by providing a solid foundation in the major areas of mathematics, an appreciation for the structures and theories of advanced mathematics, and a deeper understanding of the role of mathematics in applications. The program is structured around the three main areas of algebra, analysis, and statistics, and allows you to adapt your education to your specific career goals.'

In [1]:
from category_classifier import QuestionClassifier
classifier = QuestionClassifier()
classifier.predict('What will I learn in a math major?')

{'main_categories': ['academics'], 'sub_categories': ['academics|mathematics']}

In [4]:
classifier.predict('Is Brockport a good school?', return_probabilities=True)['main_probs']

{'support': 0.1069,
 'research-foundation': 0.0043,
 'admissions': 0.0538,
 'alumni': 0.2675,
 'life': 0.0345,
 'live': 0.0842,
 'scholarships-aid': 0.036,
 'academics': 0.231,
 'about': 0.0964,
 'graduate': 0.0685,
 'bsg': 0.0047,
 'library': 0.0122}

In [1]:
import docker

In [3]:
client = docker.from_env()
client

In [4]:
container = client.containers.get("typesense_container")
container.stop()
container.remove()

# Rework main classifier

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from torch.utils.data import TensorDataset, DataLoader
from datasets import load_dataset
import matplotlib.pyplot as plt
from copy import deepcopy
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import torch
import joblib
import json
import os

# Set default device based on whether CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = load_dataset("msaad02/categorized-data", split="train")
category_df = dataset.to_pandas()

qa_df = pd.concat([
    load_dataset("msaad02/brockport-gpt-4-qa", split="train").to_pandas(),
    load_dataset("msaad02/brockport-gpt-3.5-qa", split="train").to_pandas()
])

df = pd.merge(qa_df, category_df[["url", "category", "subcategory"]], on="url", how="left")


In [23]:
df['category'].value_counts().mean()

1935.6666666666667

In [10]:

df.dropna(subset=['category'], inplace=True)

In [12]:
df['category']

0             live
1        academics
2             life
3             life
4        academics
           ...    
23233      support
23234      support
23235      support
23236      support
23237    academics
Name: category, Length: 23228, dtype: object

In [14]:
df

,url,question,answer,category,subcategory
0,https://www2.brockport.edu/live/profiles/5447-...,Are there any areas on campus where my service...,SUNY Brockport is committed to ensuring the sa...,live,profiles
1,https://www2.brockport.edu/academics/sociology...,Are there opportunities for internships or res...,Absolutely! Many of our sociology students com...,academics,sociology
2,https://www2.brockport.edu/life/residential-li...,What can I expect during the individual interv...,"During the individual interview, you'll meet w...",life,residential-life
3,https://www2.brockport.edu/life/campus-recreat...,How can I sign up for personal training sessio...,You can sign up for personal training sessions...,life,campus-recreation
4,https://www2.brockport.edu/academics/education...,How can attending the PFSP benefit me as a new...,Attending the PFSP can significantly benefit y...,academics,educational-opportunity-program
...,...,...,...,...,...
23233,https://www2.brockport.edu/support/university-...,"What are the primary, secondary, and tertiary ...",SUNY Brockport's color palette consists of pri...,support,university-communications
23234,https://www2.brockport.edu/support/university-...,What are the guidelines for submitting content...,Content must be limited to 250 words and shoul...,support,university-communications
23235,https://www2.brockport.edu/support/human-resou...,How does SUNY Brockport promote diversity and ...,Diversity and inclusion are core values at SUN...,support,human-resources
23236,https://www2.brockport.edu/support/human-resou...,How can I apply for tuition assistance at SUNY...,You can apply for tuition assistance through p...,support,human-resources


In [15]:
# Set default device based on whether CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = load_dataset("msaad02/categorized-data", split="train")
category_df = dataset.to_pandas()

qa_df = pd.concat([
    load_dataset("msaad02/brockport-gpt-4-qa", split="train").to_pandas(),
    load_dataset("msaad02/brockport-gpt-3.5-qa", split="train").to_pandas()
])

df = pd.merge(qa_df, category_df[["url", "category", "subcategory"]], on="url", how="left")
df = df.dropna(subset=['category'])

In [16]:
df

,url,question,answer,category,subcategory
0,https://www2.brockport.edu/live/profiles/5447-...,Are there any areas on campus where my service...,SUNY Brockport is committed to ensuring the sa...,live,profiles
1,https://www2.brockport.edu/academics/sociology...,Are there opportunities for internships or res...,Absolutely! Many of our sociology students com...,academics,sociology
2,https://www2.brockport.edu/life/residential-li...,What can I expect during the individual interv...,"During the individual interview, you'll meet w...",life,residential-life
3,https://www2.brockport.edu/life/campus-recreat...,How can I sign up for personal training sessio...,You can sign up for personal training sessions...,life,campus-recreation
4,https://www2.brockport.edu/academics/education...,How can attending the PFSP benefit me as a new...,Attending the PFSP can significantly benefit y...,academics,educational-opportunity-program
...,...,...,...,...,...
23233,https://www2.brockport.edu/support/university-...,"What are the primary, secondary, and tertiary ...",SUNY Brockport's color palette consists of pri...,support,university-communications
23234,https://www2.brockport.edu/support/university-...,What are the guidelines for submitting content...,Content must be limited to 250 words and shoul...,support,university-communications
23235,https://www2.brockport.edu/support/human-resou...,How does SUNY Brockport promote diversity and ...,Diversity and inclusion are core values at SUN...,support,human-resources
23236,https://www2.brockport.edu/support/human-resou...,How can I apply for tuition assistance at SUNY...,You can apply for tuition assistance through p...,support,human-resources
